### Notebook for training the hand gesture classification

In [ ]:
#Import dependencies

import csv
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

RANDOM_SEED = 42

In [ ]:
#Path selection and definition of constant

dataset = '../assets/dataset/keypoint.csv'
model_save_path = '../assets/models'

epoche = 1000
batch_size = 128
num_classes = 6

### Dataset Reading

In [ ]:
x_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))

y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

x_train, x_test, y_train, y_test = train_test_split(x_dataset, y_dataset, train_size=0.70, random_state=RANDOM_SEED)

### Model Building

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax'),
])

model.summary()

In [ ]:
# Model chekpoint callback
# cp_callback = tf.keras.callbacks.ModelCheckpoint(
  #   model_save_path, verbose=1, save_weights_only=False)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path+'/model.h5',
    monitor='val_accuracy',
    verbose=1,
    save_weights_only=False,
    save_best_only=True,
    mode='auto',
    save_freq=1,
)

# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

### Training

In [ ]:
history = model.fit(
    x_train,
    y_train,
    epochs=epoche,
    batch_size=batch_size,
    validation_data=(x_test, y_test),
    callbacks=[cp_callback, es_callback]
)

model.save(model_save_path+'/model.h5')

In [ ]:
for key in history.history:
    print(key)

In [ ]:
# Model evaluation

val_loss, val_acc = model.evaluate(x_test, y_test, batch_size)

In [ ]:
# Loading saved model

model = tf.keras.models.load_model(model_save_path+'/model.h5')

### Confusion matrix

In [ ]:
def print_confusion_matrix(y_true, y_pred, report=True):
    labels = sorted(list(set(y_true)))
    cmx_data = confusion_matrix(y_true, y_pred, labels=labels)

    df_cmx = pd.DataFrame(cmx_data, index = labels, columns=labels)

    fig, ax = plt.subplots(figsize=(7,6))
    sns.heatmap(df_cmx, annot=True, fmt='g', square=False)
    ax.set_ylim(len(set(y_true)), 0)
    ax.set_ylabel('Estimated targets')
    ax.set_xlabel('Target value')

    plt.show()

    if report:
        print('Clasification report')
        print(classification_report(y_test, y_pred))



In [ ]:
Y_pred = model.predict(x_test)
y_pred = np.argmax(Y_pred, axis=1)

print_confusion_matrix(y_test, y_pred)